## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Matplotlib Inline Charts
%matplotlib inline

In [3]:
# Import Dependencies and Setup
import gmaps
import os
import pandas as pd
import requests

# Import API key
from config import g_key


In [4]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [15]:
# 1. Import the WeatherPy_database.csv file.

input_directory = os.path.join("..", "Weather_Database")
input_file = "WeatherPy_Database.csv"

file_to_load = os.path.join(input_directory, input_file)

city_data_df = pd.read_csv(file_to_load, index_col="City_ID")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City_ID,,,,,,,,,
0,Mataura,NZ,-46.1927,168.8643,31.75,90,100,4.00,overcast clouds
1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,77,77,10.04,broken clouds
2,Busselton,AU,-33.6500,115.3333,51.04,76,70,14.92,broken clouds
3,Tuktoyaktuk,CA,69.4541,-133.0374,51.80,93,90,17.27,light rain
4,Yinchuan,CN,38.4681,106.2731,87.84,21,35,5.99,scattered clouds


In [38]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [39]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
City_ID,,,,,,,,,
1,Puerto Ayora,EC,-0.7393,-90.3518,80.56,77,77,10.04,broken clouds
4,Yinchuan,CN,38.4681,106.2731,87.84,21,35,5.99,scattered clouds
5,Puerto Escondido,MX,15.8500,-97.0667,86.83,66,75,8.05,broken clouds
16,Borovsk,RU,55.2064,36.4861,81.10,71,0,5.41,clear sky
19,Hilo,US,19.7297,-155.0900,81.28,79,20,5.75,light rain
20,Daru,PG,-9.0763,143.2092,78.08,89,73,16.44,broken clouds
21,Ucluelet,CA,48.9329,-125.5528,78.04,75,5,4.43,clear sky
22,Atuona,PF,-9.8000,-139.0333,77.90,69,39,23.38,scattered clouds
28,Kyaukse,MM,21.6000,96.1333,82.22,71,78,7.49,broken clouds


In [40]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   277
Country                277
Lat                    277
Lng                    277
Max Temp               277
Humidity               277
Cloudiness             277
Wind Speed             277
Current Description    277
dtype: int64

In [41]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_NO_NULL_df = preferred_cities_df.dropna(how="any")
preferred_cities_NO_NULL_df.count()

City                   277
Country                277
Lat                    277
Lng                    277
Max Temp               277
Humidity               277
Cloudiness             277
Wind Speed             277
Current Description    277
dtype: int64

In [20]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_NO_NULL_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
4,Yinchuan,CN,87.84,scattered clouds,38.4681,106.2731,
5,Puerto Escondido,MX,86.83,broken clouds,15.8500,-97.0667,
16,Borovsk,RU,81.10,clear sky,55.2064,36.4861,
19,Hilo,US,81.28,light rain,19.7297,-155.0900,
20,Daru,PG,78.08,broken clouds,-9.0763,143.2092,
21,Ucluelet,CA,78.04,clear sky,48.9329,-125.5528,
22,Atuona,PF,77.90,scattered clouds,-9.8000,-139.0333,
28,Kyaukse,MM,82.22,broken clouds,21.6000,96.1333,


In [21]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [22]:
# 6b. Iterate through the hotel DataFrame.

print("Beginning Data Retrieval     ")
print("-----------------------------\n")

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Added Hotel: " + hotels["results"][0]["name"])
        
    except (IndexError):
        print("Hotel not found... skipping.")

# Indicate that Data Loading is complete.
print("\n-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------

Added Hotel: Finch Bay Galapagos Hotel
Added Hotel: JJ Inns - Yinchuan New Moon Square Branch
Added Hotel: Aldea del Bazar Hotel and Spa
Added Hotel: Dining House in Borovsk
Added Hotel: Hilo Hawaiian Hotel
Added Hotel: Daru Lodge
Added Hotel: Black Rock Oceanfront Resort
Added Hotel: Villa Enata
Added Hotel: Shwe Thit Sar Guesthouse
Added Hotel: Hampton Inn & Suites Jamestown
Added Hotel: Salinas do Maragogi All Inclusive Resort
Added Hotel: Isles Sunset Lodge
Added Hotel: Al-bustan Hotel And Restaurant.
Added Hotel: Quality Inn & Suites Banquet Center
Added Hotel: Royal Crown Hotel
Added Hotel: HOPITAL
Added Hotel: Pousada Porto De Canoas
Hotel not found... skipping.
Added Hotel: Kost Pria
Hotel not found... skipping.
Added Hotel: San Ramon Marriott
Added Hotel: Pousada Porto Praia
Added Hotel: Sheraton Kauai Coconut Beach Resort
Added Hotel: Uruna Bay Retreat
Added Hotel: Sulthan Hotel International
Added Hotel: Penginapan

Added Hotel: Sylvia Hotel
Added Hotel: Hotel Octave Maldives
Added Hotel: Mega Hotel, Miri
Added Hotel: Nuevo Hotel Coromoto
Added Hotel: Hồng Ân
Added Hotel: Ungoofaaru Inn
Added Hotel: Maitai Lapita Village Huahine
Added Hotel: hotêl doufoulougou
Added Hotel: Ponoka Stampeder Inn
Hotel not found... skipping.
Added Hotel: Hotel Villa de Aguimes
Added Hotel: Abaco Beach Resort
Added Hotel: Sanjrani House
Added Hotel: Saud Beach Resort and Hotel - 2
Added Hotel: Tarawa Boutique Hotel
Added Hotel: Plaza de Armas Bellavista de La Unión
Added Hotel: Southern Beach Hotel & Resort Okinawa
Added Hotel: Hotel Gualok
Added Hotel: waigo splash hotel
Added Hotel: Hotel Aquarius
Hotel not found... skipping.
Added Hotel: الهادي ابوشعاله سرت
Added Hotel: Motel 49
Added Hotel: Hotel Limonar
Added Hotel: Hotel del Sur
Added Hotel: Banjanay Hotel
Added Hotel: INN NEW HORIZON
Added Hotel: Thank Inn Chain Hotel gansu wuwei liangzhou district xiguan middle road
Added Hotel: Hotel Syariah CN Darussalam
Add

In [30]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
City_ID,,,,,,,
1,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
4,Yinchuan,CN,87.84,scattered clouds,38.4681,106.2731,JJ Inns - Yinchuan New Moon Square Branch
5,Puerto Escondido,MX,86.83,broken clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
16,Borovsk,RU,81.10,clear sky,55.2064,36.4861,Dining House in Borovsk
19,Hilo,US,81.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Daru,PG,78.08,broken clouds,-9.0763,143.2092,Daru Lodge
21,Ucluelet,CA,78.04,clear sky,48.9329,-125.5528,Black Rock Oceanfront Resort
22,Atuona,PF,77.90,scattered clouds,-9.8000,-139.0333,Villa Enata
28,Kyaukse,MM,82.22,broken clouds,21.6000,96.1333,Shwe Thit Sar Guesthouse


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

#clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [46]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

In [47]:
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))